# ファインチューニング
 - 参考 [Google Colab で Rinna-3.6B のLoRAファインチューニングを試す ](https://note.com/npaka/n/nc387b639e50e)
 - CUDA 12.1以上だとなんかエラー出るっぽい。（推奨は12.0.1）
 - transformers, bitsandbytes, peft, accelerateあたりを最新にしたほうが良さそう。（互換性はあるのだが、よくわからない。）

### 下準備

In [1]:
!pip install -Uqq  git+https://github.com/huggingface/peft.git
!pip install -Uqq transformers datasets accelerate bitsandbytes
!pip install sentencepiece

In [1]:
%%time

# (4) 基本パラメータの定義。
# 基本パラメータ
model_name = "rinna/japanese-gpt-neox-3.6b"
# model_name = "rinna/bilingual-gpt-neox-4b"
# model_name = "rinna/japanese-gpt2-xsmall"
dataset = "kunishou/databricks-dolly-15k-ja"
# dataset = "/home/mhiavio/work/rinna/databricks-dolly-15k-ja_1M.json"
peft_name = "lora-rinna-3.6b"
output_dir = "lora-rinna-3.6b-results"

# (6) トークナイザーの準備。
from transformers import AutoTokenizer
# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# (7) トークナイザーのスペシャルトークンの確認。
# スペシャルトークンの確認
print(tokenizer.special_tokens_map)
print("bos_token :", tokenizer.bos_token, ",", tokenizer.bos_token_id)
print("eos_token :", tokenizer.eos_token, ",", tokenizer.eos_token_id)
print("unk_token :", tokenizer.unk_token, ",", tokenizer.unk_token_id)
print("pad_token :", tokenizer.pad_token, ",", tokenizer.pad_token_id)

# (8) トークナイズ関数の定義。
CUTOFF_LEN = 256  # コンテキスト長
# トークナイズ
def tokenize(prompt, tokenizer):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
    )
    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
    }

# (9) トークナイズ関数の確認。
# トークナイズの動作確認
tokenize("hi there", tokenizer)

# (10) データセットの準備。
from datasets import load_dataset
# データセットの準備
data = load_dataset(dataset)

# (11) データセットの確認。
# データセットの確認
data["train"][5]

# (12) プロンプトテンプレートの準備。
# プロンプトテンプレートの準備
def generate_prompt(data_point):
    if data_point["input"]:
        result = f"""### 指示:
{data_point["instruction"]}

### 入力:
{data_point["input"]}

### 回答:
{data_point["output"]}"""
    else:
        result = f"""### 指示:
{data_point["instruction"]}

### 回答:
{data_point["output"]}"""

    # 改行→<NL>
    result = result.replace('\n', '<NL>')
    return result

# (13) プロンプトテンプレートの確認。
# プロンプトテンプレートの確認
print(generate_prompt(data["train"][5]))

# (14) 学習データと検証データの準備。
VAL_SET_SIZE = 2000
# 学習データと検証データの準備
train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))

# (15) モデルの準備。
from transformers import AutoModelForCausalLM
# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# (16) LoRAモデルの準備。
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType
# LoRAのパラメータ
lora_config = LoraConfig(
    r= 8, 
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
# モデルの前処理
model = prepare_model_for_int8_training(model)
# LoRAモデルの準備
model = get_peft_model(model, lora_config)
# 学習可能パラメータの確認
model.print_trainable_parameters()

# (17) トレーナーの準備。
import transformers
# eval_steps = 200
# save_steps = 200
# logging_steps = 20
eval_steps = 20
save_steps = 20
logging_steps = 2
# トレーナーの準備
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        num_train_epochs=3,
        learning_rate=3e-4,
        logging_steps=logging_steps,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        output_dir=output_dir,
        report_to="none",
        save_total_limit=3,
        push_to_hub=False,
        auto_find_batch_size=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/home/mhiavio/anaconda3/envs/rinna/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
bos_token : <s> , 2
eos_token : </s> , 3
unk_token : [UNK] , 1
pad_token : [PAD] , 0
### 指示:<NL>ステイルメイトの時に、私の方が多くの駒を持っていたら、私の勝ちですか？<NL><NL>### 入力:<NL>ステイルメイトとは、チェスにおいて、手番が回ってきたプレーヤーがチェックされておらず、合法的な手がない状態のことである。ステイルメイトの結果、引き分けとなる。終盤では、ステイルメイトは劣勢にあるプレイヤーが負けるのではなく、ゲームを引き分けることを可能にする戦術である[2]。より複雑なポジションでは、ステイルメイトはより稀で、通常は優勢側が不注意な場合にのみ成功する詐欺の形をとる[引用] ステイルメイトは終盤研究や他のチェスの問題においても共通のテーマである。<NL><NL>ステイルメイトが引き分けに統一されたのは19世紀である。それ以前は、ステイルメイトしているプレイヤーの勝利、引き分け、負けとみなされたり、反則となったり、ステイルメイトしているプレイヤーはターンを失うことになったりと、その扱いは様々であった。ステイルメイトのルールは、チェス以外のチャトランガ系ゲームごとに異なる。<NL><NL>### 回答:<NL>いいえ。<NL>ステイルメイトとは、引き分けた状態のことです。どちらがより多くの駒を捕獲したか、または優勢であるかは関係ない


Map: 100%|██████████| 2000/2000 [00:01<00:00, 1530.11 examples/s]
/home/mhiavio/anaconda3/envs/rinna/lib/python3.11/site-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 3,244,032 || all params: 3,610,489,344 || trainable%: 0.08985020286491116
CPU times: user 40.1 s, sys: 8.27 s, total: 48.4 s
Wall time: 27.9 s


In [42]:
%%time

# (18) 学習の実行。

# 学習の実行
model.config.use_cache = False
trainer.train() 
model.config.use_cache = True

# LoRAモデルの保存
trainer.model.save_pretrained(peft_name)

/home/mhiavio/anaconda3/envs/rinna/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
                                                    
  9%|▉         | 441/4881 [17:24<2:05:57,  1.70s/it]

{'loss': 8.6238, 'learning_rate': 0.00029877074370006146, 'epoch': 0.01}


                                                    
  9%|▉         | 441/4881 [18:01<2:05:57,  1.70s/it]

{'loss': 7.0019, 'learning_rate': 0.00029754148740012294, 'epoch': 0.02}


                                                    
  9%|▉         | 441/4881 [18:37<2:05:57,  1.70s/it]

...

{'loss': 2.5995, 'learning_rate': 0.00028524892440073753, 'epoch': 0.15}


                                                    
  9%|▉         | 441/4881 [25:55<2:05:57,  1.70s/it]

{'loss': 2.5593, 'learning_rate': 0.00028401966810079896, 'epoch': 0.16}


  9%|▉         | 441/4881 [26:13<4:24:05,  3.57s/it]


KeyboardInterrupt: 

In [ ]:
# (1) モデルとトークナイザーの準備。
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# LoRAモデルの準備
model = PeftModel.from_pretrained(
    model, 
    peft_name, 
    device_map="auto"
)

# 評価モード
model.eval()

In [ ]:
# (2) プロンプトテンプレートの準備。
# プロンプトテンプレートの準備
def generate_prompt(data_point):
    if data_point["input"]:
        result = f"""### 指示:
{data_point["instruction"]}

### 入力:
{data_point["input"]}

### 回答:
"""
    else:
        result = f"""### 指示:
{data_point["instruction"]}

### 回答:
"""

    # 改行→<NL>
    result = result.replace('\n', '<NL>')
    return result

In [ ]:
# (3) テキスト生成関数の定義。
# テキスト生成関数の定義
def generate(instruction,input=None,maxTokens=256):
    # 推論
    prompt = generate_prompt({'instruction':instruction,'input':input})
    input_ids = tokenizer(prompt, 
        return_tensors="pt", 
        truncation=True, 
        add_special_tokens=False).input_ids.cuda()
    outputs = model.generate(
        input_ids=input_ids, 
        max_new_tokens=maxTokens, 
        do_sample=True,
        temperature=0.7, 
        top_p=0.75, 
        top_k=40,         
        no_repeat_ngram_size=2,
    )
    outputs = outputs[0].tolist()
    print(tokenizer.decode(outputs))

    # EOSトークンにヒットしたらデコード完了
    if tokenizer.eos_token_id in outputs:
        eos_index = outputs.index(tokenizer.eos_token_id)
        decoded = tokenizer.decode(outputs[:eos_index])

        # レスポンス内容のみ抽出
        sentinel = "### 回答:"
        sentinelLoc = decoded.find(sentinel)
        if sentinelLoc >= 0:
            result = decoded[sentinelLoc+len(sentinel):]
            print(result.replace("<NL>", "\n"))  # <NL>→改行
        else:
            print('Warning: Expected prompt template to be emitted.  Ignoring output.')
    else:
        print('Warning: no <eos> detected ignoring output')

In [ ]:
# (4) 推論の実行。
generate("自然言語処理とは？")